#Transfer learning using Universal Sentence Encoder

### Dataset Description

### Emotions Dataset

The dataset contains 16000 train values and 2000 test values, each containing two features

**Objective** The objective is to predict the emotions bases on the text messages

**Columns**

1. Text- The text messages

2. Label- The different emotions

### 1. Importing packages and Universal Sentence Encoder

In [ ]:
# Importing packages

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import keras.layers as layers
from keras.models import Model
from keras import backend as K
np.random.seed(10)


# The Universal Sentence Encoder module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"


# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### 2. Assigning messages using word, sentence, paragraphs

In [ ]:
# Compute a representation for each message, showing various lengths supported.
word = "Levitization"
sentence = "There are two types of Pain in this world: Pain that hurts you, and Pain that changes you"
paragraphs = (
    "Everyone is a genius. "
    "But if you judge a fish on its ability to climb a tree," 
     "it will live its whole life believing it is stupid. ")
messages = [word, sentence, paragraphs]

### 3. Performing Universal Sentence Encoding on the messages

In [ ]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

# Embed the messages using Universal Sentence Encoder

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))
    
# Print all the messages with its embedding and size  
  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: Levitization
Embedding size: 512
Embedding: [-0.01299950573593378, -0.0030302153900265694, 0.024302732199430466, ...]

Message: There are two types of Pain in this world: Pain that hurts you, and Pain that changes you
Embedding size: 512
Embedding: [0.031403250992298126, 0.04196187108755112, 0.014515469782054424, ...]

Message: Everyone is a genius. But if you judge a fish on its ability to climb a tree,it will live its whole life believing it is stupid. 
Embedding size: 512
Embedding: [0.03774511069059372, 0.07424656301736832, 0.06033611297607422, ...]



In [ ]:
#size of embedding
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value
embed_size

512

### 4. Reading the dataset  and processing

In [ ]:
# Reading the datasets
train = pd.read_csv('train.txt',sep=';',names=['text','label'])
test = pd.read_csv('test.txt',sep=';',names=['text','label'])

In [ ]:
test.shape

(2000, 2)

In [ ]:
train.shape

(16000, 2)

In [ ]:
# Getting the head of the datasets
print('Train data',train.head())
print('Test data',test.head())

Train data                                                 text    label
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger
Test data                                                 text    label
0  im feeling rather rotten so im not very ambiti...  sadness
1          im updating my blog because i feel shitty  sadness
2  i never make her separate from me because i do...  sadness
3  i left with my bouquet of red and yellow tulip...      joy
4    i was feeling a little vain when i did this one  sadness


In [ ]:
train['label'] = train['label'].astype('category')

In [ ]:
#Number of categories in dataset
category_counts = len(train.label.cat.categories)
category_counts

6

In [ ]:
#Convert train text to  an array and labels as get_dummies

train_text = train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = np.asarray(pd.get_dummies(train.label), dtype = np.int8)

In [ ]:
#Convert train text to  an array and labels as get_dummies

test_text = test['text'].tolist()
test_text = np.array(test_text,dtype=object)[:,np.newaxis]
test_label = np.asarray(pd.get_dummies(test.label),dtype=np.int8)

### 5. Building the model using Universal Sentence Encoder

In [ ]:
#Creating function for Universal Embedding 
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
# Model Building
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(UniversalEmbedding, output_shape=(embed_size,))(input_text)
dense = layers.Dense(512, activation='relu')(embedding)
pred = layers.Dense(category_counts, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 265,734
Trainable params: 265,734
Non-trainable params: 0
_________________________________________________________________


### 6. Training the model

In [ ]:
# Training the model

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,
            batch_size=32)
  model.save_weights('./model.h5')

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Train on 16000 samples, validate on 2000 samples
Epoch 1/10
16000/16000 [==============================] - 967s 60ms/step - loss: 1.1157 - accuracy: 0.5870 - val_loss: 1.0059 - val_accuracy: 0.6275
Epoch 2/10
16000/16000 [==============================] - 954s 60ms/step - loss: 0.9804 - accuracy: 0.6331 - val_loss: 0.9795 - val_accuracy: 0.6235
Epoch 3/10
16000/16000 [==============================] - 955s 60ms/step - loss: 0.9215 - accuracy: 0.6573 - val_loss: 0.9396 - val_accuracy: 0.6555
Epoch 4/10
16000/16000 [==============================] - 955s 60ms/step - loss: 0.8636 - accuracy: 0.6798 - val_loss: 0.9214 - val_accuracy: 0.6450
Epoch 5/10
16000/16000 [==============================] - 949s 59ms/step - loss: 0.8071 - accuracy: 0.6991 - val_loss: 0.8922 - val_accuracy: 0.6530
Epoch 6/10
16000/16000 [==============================] - 956s 60ms/step - loss: 0.7549 - accuracy: 0.7222 - val_loss: 0.8834 - val_accuracy: 0.6665
Epoch 7/10
16000/16000 [==============================] -

### 7. Testing the model

In [ ]:
# Testing the model

new_text = ['i just feel like all my efforts are in vain and a waste of time',
            'i feel as if i am the beloved preparing herself for the wedding',
           'i see the starlight caress your hair no more feel the tender kisses we used to share i close my eyes and clearly my heart remembers a thousand good byes could never put out the embers',
           'i feel and im amazed of how often i think i need to save the world',
            'i start to lose that sense of independence in that i feel a lot more hesitant to do things']

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text, batch_size=32)

categories = train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BCED47E488>
Traceback (most recent call last):
  File "C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "C:\Users\sudha\Anaconda3\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


['sadness', 'joy', 'love', 'surprise', 'fear']
